In [1]:
#!M
# !pip install ../input/python-datatable/datatable-0.11.0-cp37-cp37m-manylinux2010_x86_64.whl > /dev/null 2>&1
# %pip install --upgrade pip
# %pip install -r /home/jupyter/work/resources/riiidNew/requirements.txt --upgrade
# %pip install scipy  --ignore-installed scipy --upgrade
# %pip install pyarrow==0.17.1
# %pip install bayesian-optimization
print("ok")

ok


In [2]:
#!L
# if str(Path.home()) == "/root":  # DATASPHERE
import sys
import gc
from pathlib import Path
sys.path.append(str(Path.cwd()/'riiidNew'))
from pathlib import Path
import numpy as np
import pandas as pd
# import pyarrow
from collections import defaultdict
from catboost import CatBoostClassifier, Pool
from preprocess import preprocess_train_data
from catboost_bayesian_search import bayesian_catboost_searchCV, param_adjust_dtypes
import config

In [3]:
# Error handling, ignore all
np.seterr(divide = 'ignore', invalid = 'ignore')


if str(Path.home()) == "/home/sergey":   # Home computer
    data_path = Path.home()/"mnt"/"4.5Tb"/"Downloads"/"riiidCVdata"
    kaggle_path = Path.cwd()/"kaggle_tmp/"
    questions_df = pd.read_csv(data_path/'questions.csv', usecols = [0, 3], dtype = {'question_id': 'int16', 'part': 'int8'})
    # test_df = pd.read_feather(data_path/'cv2_valid_1e4.feather', columns=config.dtypes.keys())
    train_df = pd.read_feather(data_path/'cv2_train_1e5.feather', columns=config.dtypes.keys())
    # test_df = pd.read_feather(data_path/"cv1_valid_1e4.feather", columns=config.dtypes.keys())
    # train_df = pd.read_feather(data_path/"cv1_train.feather", columns=config.dtypes.keys())
    # test_df = pd.read_feather(data_path/"cv1_valid.feather", columns=config.dtypes.keys())

elif str(Path.home()) == "/root":   # DATASPHERE

    path = Path.cwd()/"riiidNew"/"data1"
    kaggle_path = Path.cwd()/"riiidNew"/"kaggle"
    questions_df = pd.read_csv(path/'questions.csv', usecols = [0, 3], dtype = {'question_id': 'int16', 'part': 'int8'})
    train_df = pd.read_feather(path/"train.feather", columns=config.dtypes.keys())
    # train_df = pd.read_feather(path/"train_5e6.feather", columns=config.dtypes.keys())
    # train_df = pd.read_feather(path/"train_1e5.feather", columns=config.dtypes.keys())
    # train_df = pd.read_pickle(path/'cv2_train.pickle.zip').astype(config.dtypes, errors="ignore")
    # test_df = pd.read_pickle(path/"cv2_valid.pickle.zip").astype(config.dtypes, errors="ignore")

elif str(Path.home()) == "/home/riiid":   # Virtual machine ycloud

    path = Path.cwd()/"data"
    kaggle_path = Path.cwd()/"kaggle"
    questions_df = pd.read_csv(path/'questions.csv', usecols = [0, 3], dtype = {'question_id': 'int16', 'part': 'int8'})
    # train_df = pd.read_feather(path/"train.feather", columns=config.dtypes.keys())
    train_df = pd.read_feather(path/"train_5e6.feather", columns=config.dtypes.keys())
#     train_df = pd.read_feather(path/"train_1e5.feather", columns=config.dtypes.keys())
#     train_df = train_df.iloc[:100]

print(f"train_df shape = {train_df.shape}")

train_df shape = (100, 10)


In [4]:
# Preprocess for Bayesian CV
train_user_agg, train_content_agg, train_df = preprocess_train_data(train_df, questions_df, config.target, config.dtypes)
# test_user_agg, test_content_agg, test_df = preprocess_train_data(test_df, questions_df, config.target, config.dtypes)
print("Preprocess for CV ok")

# Training and validating data
train_set = Pool(train_df[config.features], label = train_df[config.target], cat_features=config.cat_features)
# val_set = Pool(test_df[features], label = test_df[target])

print("optimizer_maxCV ..")
optimizer_maxCV = bayesian_catboost_searchCV(
    train_set,
    prior_params=config.prior_params,
    pds=config.pds, pds_dtypes=config.pds_dtypes,
    init_points=5, n_iter=15, verbose=False,
)
print("optimizer_maxCV ..")
print(optimizer_maxCV)



Preprocess for CV ok
optimizer_maxCV ..
0:	learn: 0.9887798	test: 0.8683862	best: 0.8683862 (0)	total: 116ms	remaining: 2.2s
19:	learn: 0.9990650	test: 0.9903199	best: 0.9903199 (16)	total: 1.58s	remaining: 0us
0:	learn: 0.9970980	test: 0.9343434	best: 0.9343434 (0)	total: 104ms	remaining: 1.98s
19:	learn: 1.0000000	test: 0.9949495	best: 0.9974747 (11)	total: 1.48s	remaining: 0us
optimizer_maxCV ..
{'target': 0.9974747474747474, 'params': {'bagging_temperature': 2.334518955198374, 'border_count': 161.20576428639973, 'depth': 8.157022413275127, 'l2_leaf_reg': 382.8994663102174, 'max_leaves': 116.8199945214107}}


In [5]:
del train_df
gc.collect()

22

In [6]:
# Obtain new train_df and validation test_df
# train_df = pd.read_feather(path/"cv2_train.feather", columns=config.dtypes.keys())
train_df = pd.read_feather(path/"train.feather", columns=config.dtypes.keys())


# Preprocess for model fitting
train_user_agg, train_content_agg, train_df = preprocess_train_data(train_df, questions_df, config.target, config.dtypes)
print("Preprocess for model fitting ok")

# Ratio is 6/24 = 25%
test_df = train_df.groupby('user_id').tail(6)
train_df.drop(test_df.index, inplace = True)

# Training and validating data
train_set = Pool(train_df[config.features], label = train_df[config.target], cat_features=config.cat_features)
val_set = Pool(test_df[config.features], label = test_df[config.target], cat_features=config.cat_features)


# val_set = Pool(valid_df[features], label = valid_df[target])
params = param_adjust_dtypes(config.prior_params,
                             config.pds_dtypes,
                             optimizer_maxCV["params"])
params["learning_rate"] = 4e-3
params['iterations'] = 20000

print("params")
print(params)

model = CatBoostClassifier(**params)

# Fitting
model.fit(train_set, eval_set = val_set, use_best_model = True)
print(f"kaggle_path {kaggle_path}")

model.save_model(f"{kaggle_path/'catboost.model'}")

print("model.get_best_score()")
print(model.get_best_score())

Preprocess for model fitting ok
params
{'loss_function': 'Logloss', 'eval_metric': 'AUC', 'custom_metric': 'AUC:hints=skip_train~false', 'task_type': 'CPU', 'grow_policy': 'Lossguide', 'iterations': 20000, 'learning_rate': 0.004, 'random_seed': 0, 'bootstrap_type': 'Bayesian', 'l2_leaf_reg': 382.8994663102174, 'depth': 8, 'max_leaves': 116, 'border_count': 161, 'verbose': 250, 'od_type': 'Iter', 'od_wait': 50, 'bagging_temperature': 2.334518955198374}
0:	learn: 0.7815791	test: 0.7415435	best: 0.7415435 (0)	total: 24.8ms	remaining: 8m 16s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.7627364459
bestIteration = 17

Shrink model to first 18 iterations.
kaggle_path /home/riiid/riiidNew/kaggle
model.get_best_score()
{'learn': {'Logloss': 0.641919752846583, 'AUC': 0.806492057930675}, 'validation': {'Logloss': 0.6505131918826301, 'AUC': 0.76273644587254}}


In [7]:
from pathlib import Path
Path.home()

PosixPath('/home/riiid')

Inference

In [8]:
#!M
import pickle

user_sum_dict = train_user_agg['sum'].astype('int32').to_dict(defaultdict(int))
user_count_dict = train_user_agg['count'].astype('int32').to_dict(defaultdict(int))
content_sum_dict = train_content_agg['sum'].astype('int32').to_dict(defaultdict(int))
content_count_dict = train_content_agg['count'].astype('int32').to_dict(defaultdict(int))

for filename, dic in zip(["user_sum_dict", "user_count_dict", "content_sum_dict", "content_count_dict"],
                         [user_sum_dict, user_count_dict, content_sum_dict, content_count_dict]):
    with open(f'{kaggle_path}/{filename}.pickle', 'wb') as handle:
        pickle.dump(dic, handle)

In [9]:
train_df.head()
del train_df
gc.collect()

79

In [10]:
%%time
validaten_flg = True
if validaten_flg:
    from emulator import Iter_Valid
    iter_test = Iter_Valid(test_df,max_user=1000)
    predicted = []
    def set_predict(df):
        predicted.append(df)
else:
    import riiideducation
    env = riiideducation.make_env()
    iter_test = env.iter_test()
    set_predict = env.predict

CPU times: user 10.6 ms, sys: 686 µs, total: 11.2 ms
Wall time: 27.1 ms


In [11]:
%%time

prior_test_df = None
for (test_df, sample_prediction_df) in iter_test:
    if prior_test_df is not None:
        prior_test_df[config.target] = eval(test_df['prior_group_answers_correct'].iloc[0])
        prior_test_df = prior_test_df[prior_test_df[config.target] != -1].reset_index(drop = True)

        user_ids = prior_test_df['user_id'].values
        content_ids = prior_test_df['content_id'].values
        targets = prior_test_df[config.target].values

        # for user_id, content_id, answered_correctly in prior_test_df[["user_id", "content_id", target]].values:
        for user_id, content_id, answered_correctly in zip(user_ids, content_ids, targets):
            user_sum_dict[user_id] += answered_correctly
            user_count_dict[user_id] += 1
            content_sum_dict[content_id] += answered_correctly
            content_count_dict[content_id] += 1

    prior_test_df = test_df.copy()

    test_df = test_df[test_df['content_type_id'] == 0].reset_index(drop = True)
    test_df.drop(labels="part", axis=1, inplace=True)
    test_df.content_id = test_df.content_id.astype(int)

    test_df = pd.merge(test_df, questions_df, left_on = 'content_id', right_on = 'question_id', how = 'left')
    test_df['prior_question_had_explanation'] = test_df['prior_question_had_explanation'].fillna(False).astype('bool')
    user_sum = np.zeros(len(test_df), dtype = np.int32)
    user_count = np.zeros(len(test_df), dtype = np.int32)
    content_sum = np.zeros(len(test_df), dtype = np.int32)
    content_count = np.zeros(len(test_df), dtype = np.int32)

    for i, (user_id, content_id) in enumerate(zip(test_df['user_id'].values, test_df['content_id'].values)):
        user_sum[i] = user_sum_dict[user_id]
        user_count[i] = user_count_dict[user_id]
        content_sum[i] = content_sum_dict[content_id]
        content_count[i] = content_count_dict[content_id]

    test_df['user_correctness'] = user_sum / user_count
    test_df['content_count'] = content_count
    test_df['content_id'] = content_sum / content_count
    test_df[config.target] = model.predict_proba(test_df[config.features])[:,1]
    set_predict(test_df[['row_id', config.target]])

CPU times: user 26.8 s, sys: 602 ms, total: 27.4 s
Wall time: 26.4 s
